In [ ]:
pip install optuna transformers torch tensorflow

In [ ]:
pip install tensorflow

In [ ]:
import pandas as pd
df = pd.read_excel("/content/Final_student_tickting.xlsx")
df["Genre"].value_counts()

,count
Genre,
Academic Support and Resources,1097
Online learning,973
Food and Cantines,942
International student experiences,926
Athletics and sports,896
Health and Well-being Support,890
Activities and Travelling,882
Student Affairs,871
Financial Support,851


In [ ]:
def sample_per_genre(df, genre_column="Genre", sample_size=125, seed=42):
    # Filter only genres that have at least `sample_size` rows
    genre_counts = df[genre_column].value_counts()
    valid_genres = genre_counts[genre_counts >= sample_size].index

    # Sample from each genre
    sampled_df = (
        df[df[genre_column].isin(valid_genres)]
        .groupby(genre_column)
        .sample(n=sample_size, random_state=seed)
        .reset_index(drop=True)
    )

    return sampled_df


In [ ]:
df = pd.read_excel("/content/Final_student_tickting.xlsx")


df = sample_per_genre(df)

# Check the result
print(df['Genre'].value_counts())
print(df.shape)


df=df.sample(frac=1,random_state=42).reset_index(drop=True)

df.head(15)

Genre
Academic Support and Resources       125
Activities and Travelling            125
Athletics and sports                 125
Career opportunities                 125
Financial Support                    125
Food and Cantines                    125
Health and Well-being Support        125
Housing and Transportation           125
International student experiences    125
Online learning                      125
Student Affairs                      125
Name: count, dtype: int64
(1375, 35)


,Unnamed: 0,Genre,Reports,Age,Year,Count,Gender,Nationality,Resolution,Student Name,...,solution_vader_score,solution_sentiment_score,response_missing,resolution_missing,response_time_filled,days_to_resolution_filled,time_score,sentiment_improvement,composite_score,degree_of_satisfaction
0,4756,Financial Support,I'm worried about how much student loan debt I...,26,Second,1,Male,Egypt,Encourage companies to offer remote and hybrid...,James Williams,...,0.7096,0.000000,1,0,0,0.299948,0.910016,0.341030,0.664521,Satisfied
1,9569,Academic Support and Resources,Limited access to academic resources is induce...,19,First,1,Female,Egypt,Launch a university-sponsored online research ...,Shannon Lawrence,...,0.2960,0.679726,1,1,0,0.032040,0.992310,0.592205,0.457816,Neutral
2,2389,Housing and Transportation,I wish the university would provide more affor...,16,Second,1,Male,South Africa,Develop affordable housing options on or near ...,Leonard Price,...,0.1027,0.000000,1,0,0,0.284058,0.914782,-0.004730,0.433548,Neutral
3,2486,Food and Cantines,"The campus coffee shop is always crowded, and ...",20,Third,1,Male,Nigeria,Increase staffing at the campus coffee stag du...,Robin Hancock,...,0.3182,0.000000,1,0,0,0.006121,0.998164,0.087090,0.592810,Neutral
4,5650,Activities and Travelling,I wish there were more opportunities to connec...,26,Third,1,Male,United Kingdom,Create interest-based clubs and organizations ...,Mark Morgan,...,0.8225,0.685460,1,1,0,0.032040,0.992310,0.672266,0.499834,Neutral
5,7518,Financial Support,"I'm struggling to make close meet, even with f...",19,Second,1,Male,Egypt,produce a collaborative job search strategy wh...,Mark Hill,...,0.7430,0.000000,1,1,0,0.032040,0.992310,0.339350,0.114656,Very Dissatisfied
6,7501,Career opportunities,"Despite my efforts to network, I have not been...",23,Second,1,Female,Egypt,Strengthen alumni networking initiatives to fa...,Abigail Acosta,...,0.3182,0.603946,1,1,0,0.032040,0.992310,0.429708,0.427067,Neutral
7,4585,Financial Support,I wish there were more opportunities for scho...,25,Third,1,Male,Egypt,enlarge meal assistance programs for students ...,Robert Jones,...,-0.4215,0.000000,1,0,0,0.107971,0.967609,-0.236610,0.398739,Neutral
8,2894,Activities and Travelling,I love admit weekend trips to nearby cities an...,21,Third,1,Female,United States,Organize affordable weekend activate to nearby...,Hayley Melton,...,0.4019,0.701056,1,0,0,0.044543,0.986637,0.493603,0.711457,Satisfied
9,4328,Activities and Travelling,I really enjoy exploring new vicinity and tryi...,20,Third,1,Male,China,Establish a foodie club to help students explo...,Vincent Cameron,...,0.5994,0.501200,1,1,0,0.032040,0.992310,0.460580,0.418328,Neutral


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tqdm.auto import tqdm
import numpy as np
import random

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()


X = df['Reports'].values
y = df['Genre'].values

# Train-Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Label Encoding
le = LabelEncoder()
y_train_encoder = le.fit_transform(y_train)
y_val_encoder = le.transform(y_val)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize data
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(X_val.tolist(), truncation=True, padding=True, max_length=128)

# Create PyTorch datasets
train_dataset = TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(y_train_encoder)
)
val_dataset = TensorDataset(
    torch.tensor(val_encodings['input_ids']),
    torch.tensor(val_encodings['attention_mask']),
    torch.tensor(y_val_encoder)
)

# Hyperparameters (optimized for BERT)
best_params = {
    "lr": 2e-5,
    "batch_size": 16,
    "epochs": 4,  # Slightly increased for better convergence
    "weight_decay": 0.01,
    "warmup_steps": 100,
    "grad_clip": 1.0  # Gradient clipping
}

# Initialize model with dropout
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(le.classes_),
    hidden_dropout_prob=0.2,  # Increased dropout
    attention_probs_dropout_prob=0.2
)
model.to(device)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"])

# Optimizer and scheduler
optimizer = AdamW(model.parameters(),
                 lr=best_params["lr"],
                 weight_decay=best_params["weight_decay"])
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=best_params["warmup_steps"],
    num_training_steps=len(train_loader) * best_params["epochs"]
)

# Training loop
best_val_accuracy = 0.0
for epoch in range(best_params["epochs"]):
    print(f"\n{'='*30}")
    print(f"Epoch {epoch+1}/{best_params['epochs']}")
    print(f"{'='*30}")

    # Training
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc="Training", position=0, leave=True)

    for batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        outputs = model(input_ids,
                       attention_mask=attention_mask,
                       labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), best_params["grad_clip"])
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_train_loss = epoch_loss / len(train_loader)
    print(f"\nTrain Loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        val_bar = tqdm(val_loader, desc="Validation", position=0, leave=True)
        for batch in val_bar:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            outputs = model(input_ids,
                          attention_mask=attention_mask,
                          labels=labels)

            val_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(val_loader)

    print(f"\nValidation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=le.classes_))

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_accuracy': val_accuracy,
            'epoch': epoch
        }, "best_model.pt")
        print(f"\n🔥 New best model saved (Accuracy: {val_accuracy:.4f})")

print(f"\n{'='*50}")
print(f"Training Complete | Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"{'='*50}")

# Load best model for inference
checkpoint = torch.load("best_model.pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/4


Training:   0%|          | 0/50 [00:00<?, ?it/s]


Train Loss: 2.3875


Validation:   0%|          | 0/13 [00:00<?, ?it/s]

...


Validation Loss: 0.8336
Validation Accuracy: 0.8700

Classification Report:
                                   precision    recall  f1-score   support

   Academic Support and Resources       0.76      1.00      0.86        48
        Activities and Travelling       0.00      0.00      0.00        12
             Athletics and sports       0.94      0.94      0.94        16
             Career opportunities       1.00      1.00      1.00        12
                Financial Support       0.87      0.87      0.87        15
                Food and Cantines       0.94      1.00      0.97        30
    Health and Well-being Support       1.00      0.75      0.86        12
       Housing and Transportation       0.81      1.00      0.90        13
International student experiences       0.84      0.80      0.82        20
                  Online learning       1.00      1.00      1.00        18
                  Student Affairs       0.00      0.00      0.00         4

                     

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



🔥 New best model saved (Accuracy: 0.8700)

Training Complete | Best Validation Accuracy: 0.8700


<ipython-input-7-1ce878445a1e>:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("best_model.pt")


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

Predict


In [ ]:
def predict_genre(text, model, tokenizer, label_encoder, max_length=128):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=max_length)

    # Move to GPU if available
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()

    return label_encoder.inverse_transform([prediction])[0]


In [ ]:

sample_text = "i has an issue of my lunch i always hungary"

# Predict genre
predicted_genre = predict_genre(sample_text, model, tokenizer, le)
print(f"Predicted Genre: {predicted_genre}")


Predicted Genre: Food and Cantines


In [ ]:
len(df)

1375

In [ ]:
df.head(15)

,Unnamed: 0,Genre,Reports,Age,Year,Count,Gender,Nationality,Resolution,Student Name,...,solution_vader_score,solution_sentiment_score,response_missing,resolution_missing,response_time_filled,days_to_resolution_filled,time_score,sentiment_improvement,composite_score,degree_of_satisfaction
0,4756,Financial Support,I'm worried about how much student loan debt I...,26,Second,1,Male,Egypt,Encourage companies to offer remote and hybrid...,James Williams,...,0.7096,0.000000,1,0,0,0.299948,0.910016,0.341030,0.664521,Satisfied
1,9569,Academic Support and Resources,Limited access to academic resources is induce...,19,First,1,Female,Egypt,Launch a university-sponsored online research ...,Shannon Lawrence,...,0.2960,0.679726,1,1,0,0.032040,0.992310,0.592205,0.457816,Neutral
2,2389,Housing and Transportation,I wish the university would provide more affor...,16,Second,1,Male,South Africa,Develop affordable housing options on or near ...,Leonard Price,...,0.1027,0.000000,1,0,0,0.284058,0.914782,-0.004730,0.433548,Neutral
3,2486,Food and Cantines,"The campus coffee shop is always crowded, and ...",20,Third,1,Male,Nigeria,Increase staffing at the campus coffee stag du...,Robin Hancock,...,0.3182,0.000000,1,0,0,0.006121,0.998164,0.087090,0.592810,Neutral
4,5650,Activities and Travelling,I wish there were more opportunities to connec...,26,Third,1,Male,United Kingdom,Create interest-based clubs and organizations ...,Mark Morgan,...,0.8225,0.685460,1,1,0,0.032040,0.992310,0.672266,0.499834,Neutral
5,7518,Financial Support,"I'm struggling to make close meet, even with f...",19,Second,1,Male,Egypt,produce a collaborative job search strategy wh...,Mark Hill,...,0.7430,0.000000,1,1,0,0.032040,0.992310,0.339350,0.114656,Very Dissatisfied
6,7501,Career opportunities,"Despite my efforts to network, I have not been...",23,Second,1,Female,Egypt,Strengthen alumni networking initiatives to fa...,Abigail Acosta,...,0.3182,0.603946,1,1,0,0.032040,0.992310,0.429708,0.427067,Neutral
7,4585,Financial Support,I wish there were more opportunities for scho...,25,Third,1,Male,Egypt,enlarge meal assistance programs for students ...,Robert Jones,...,-0.4215,0.000000,1,0,0,0.107971,0.967609,-0.236610,0.398739,Neutral
8,2894,Activities and Travelling,I love admit weekend trips to nearby cities an...,21,Third,1,Female,United States,Organize affordable weekend activate to nearby...,Hayley Melton,...,0.4019,0.701056,1,0,0,0.044543,0.986637,0.493603,0.711457,Satisfied
9,4328,Activities and Travelling,I really enjoy exploring new vicinity and tryi...,20,Third,1,Male,China,Establish a foodie club to help students explo...,Vincent Cameron,...,0.5994,0.501200,1,1,0,0.032040,0.992310,0.460580,0.418328,Neutral


In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np
import random
from tqdm.auto import tqdm
import joblib
import os
from datetime import datetime

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()


X = df['Reports'].values
y = df['Genre'].values

# --- Train-Validation Split ---
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # Stratify for class balance
)

# --- Label Encoding ---
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)

# Save the label encoder for inference
os.makedirs("model_assets", exist_ok=True)
joblib.dump(le, "model_assets/label_encoder.joblib")

# --- Tokenizer ---
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize with explicit settings
train_encodings = tokenizer(
    X_train.tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors="pt"
)
val_encodings = tokenizer(
    X_val.tolist(),
    truncation=True,
    padding='max_length',
    max_length=128,
    return_tensors="pt"
)

# --- Datasets ---
train_dataset = TensorDataset(
    train_encodings['input_ids'],
    train_encodings['attention_mask'],
    torch.tensor(y_train_encoded)
)
val_dataset = TensorDataset(
    val_encodings['input_ids'],
    val_encodings['attention_mask'],
    torch.tensor(y_val_encoded)
)

# --- Hyperparameters ---
best_params = {
    "lr": 2e-5,
    "batch_size": 16,
    "epochs": 5,
    "weight_decay": 0.01,
    "warmup_steps": 100,
    "grad_clip": 1.0
}

# --- Model Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(le.classes_),
    hidden_dropout_prob=0.2,
    attention_probs_dropout_prob=0.2
)
model.to(device)

# --- DataLoaders ---
train_loader = DataLoader(
    train_dataset,
    batch_size=best_params["batch_size"],
    shuffle=True,
    pin_memory=True  # Faster data transfer to GPU
)
val_loader = DataLoader(
    val_dataset,
    batch_size=best_params["batch_size"],
    pin_memory=True
)

# --- Optimizer & Scheduler ---
optimizer = AdamW(
    model.parameters(),
    lr=best_params["lr"],
    weight_decay=best_params["weight_decay"]
)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=best_params["warmup_steps"],
    num_training_steps=len(train_loader) * best_params["epochs"]
)

# --- Training Loop ---
best_val_accuracy = 0.0
training_stats = []  # Track metrics per epoch

for epoch in range(best_params["epochs"]):
    print(f"\n{'='*30}")
    print(f"Epoch {epoch+1}/{best_params['epochs']}")
    print(f"{'='*30}")

    # Training Phase
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc="Training", leave=True)

    for batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]

        # Forward pass
        outputs = model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), best_params["grad_clip"])
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

    avg_train_loss = epoch_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation", leave=True):
            input_ids, attention_mask, labels = [b.to(device) for b in batch]

            outputs = model(
                input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            val_loss += outputs.loss.item()
            preds = torch.argmax(outputs.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(val_loader)

    # Save epoch stats
    training_stats.append({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "val_loss": avg_val_loss,
        "val_accuracy": val_accuracy
    })

    # Print metrics
    print(f"\nTrain Loss: {avg_train_loss:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.4f}")
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=le.classes_))

    # Save best model
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save({
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "val_accuracy": val_accuracy,
            "training_stats": training_stats
        }, "model_assets/best_model.pt")
        print(f"🔥 New best model saved (Accuracy: {val_accuracy:.4f})")

# --- Final Summary ---
print(f"\n{'='*50}")
print(f"Training Complete | Best Val Accuracy: {best_val_accuracy:.4f}")
print(f"{'='*50}")

# Save tokenizer and config for inference
tokenizer.save_pretrained("model_assets")
model.config.save_pretrained("model_assets")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Validation:   0%|          | 0/18 [00:00<?, ?it/s]


Train Loss: 0.5466
Val Loss: 0.3583 | Val Accuracy: 0.9782

Classification Report:
                                   precision    recall  f1-score   support

   Academic Support and Resources       1.00      1.00      1.00        25
        Activities and Travelling       1.00      0.92      0.96        25
             Athletics and sports       1.00      1.00      1.00        25
             Career opportunities       0.96      1.00      0.98        25
                Financial Support       1.00      0.92      0.96        25
                Food and Cantines       1.00      1.00      1.00        25
    Health and Well-being Support       0.93      1.00      0.96        25
       Housing and Transportation       0.93      1.00      0.96        25
International student experiences       0.96      1.00      0.98        25
                  Online learning       1.00      1.00      1.00        25
                  Student Affairs       1.00      0.92      0.96        25

              

Training:   0%|          | 0/69 [00:00<?, ?it/s]

Validation:   0%|          | 0/18 [00:00<?, ?it/s]


Train Loss: 0.3567
Val Loss: 0.2794 | Val Accuracy: 0.9818

Classification Report:
                                   precision    recall  f1-score   support

   Academic Support and Resources       1.00      1.00      1.00        25
        Activities and Travelling       1.00      0.92      0.96        25
             Athletics and sports       1.00      1.00      1.00        25
             Career opportunities       0.96      1.00      0.98        25
                Financial Support       1.00      0.92      0.96        25
                Food and Cantines       1.00      1.00      1.00        25
    Health and Well-being Support       0.96      1.00      0.98        25
       Housing and Transportation       0.93      1.00      0.96        25
International student experiences       0.96      1.00      0.98        25
                  Online learning       1.00      1.00      1.00        25
                  Student Affairs       1.00      0.96      0.98        25

              

In [ ]:
# --- Inference Example ---
def predict(text, model, tokenizer, label_encoder):
    """Helper function for inference."""
    model.eval()
    inputs = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()

    return label_encoder.inverse_transform(preds)[0]


loaded_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(le.classes_)).to(device)
loaded_model.load_state_dict(torch.load("model_assets/best_model.pt")["model_state_dict"]) # Load the saved model weights
le = joblib.load("model_assets/label_encoder.joblib")
print(predict("i have an issue at the internet access", loaded_model, tokenizer, le))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-38-0897c9285755>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We rec

Online learning


In [ ]:
import torch
import joblib
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForSequenceClassification

# Pre-load assets once (outside the prediction function)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
le = joblib.load("model_assets/label_encoder.joblib")

# Load model with optimizations
loaded_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(le.classes_)
).to(device)
loaded_model.load_state_dict(torch.load("model_assets/best_model.pt")["model_state_dict"])

# Apply optimizations
loaded_model.eval()
torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuner
if torch.cuda.is_available():
    loaded_model = torch.compile(loaded_model)  # PyTorch 2.0 compiler

def predict(text, model=loaded_model, tokenizer=tokenizer, label_encoder=le):
    """Returns prediction with confidence score"""
    with torch.no_grad(), torch.inference_mode():
        inputs = tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors="pt"
        ).to(device)

        # Use fused kernels where available
        with torch.backends.cuda.sdp_kernel(enable_flash=True):
            outputs = model(**inputs)

        # Get probabilities and prediction
        probs = softmax(outputs.logits, dim=1)
        confidence, pred_idx = torch.max(probs, dim=1)

        return {
            'prediction': label_encoder.inverse_transform(pred_idx.cpu().numpy())[0],
            'confidence': round(confidence.item(), 4)  # Rounded to 4 decimal places
        }

def predict_batch(texts, batch_size=32):
    """Batch prediction with confidence scores"""
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="pt",
        add_special_tokens=True
    )

    dataset = torch.utils.data.TensorDataset(
        encodings['input_ids'],
        encodings['attention_mask']
    )

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

    results = []
    with torch.no_grad(), torch.inference_mode():
        for batch in loader:
            inputs = {k: v.to(device) for k, v in zip(['input_ids', 'attention_mask'], batch)}
            with torch.backends.cuda.sdp_kernel(enable_flash=True):
                outputs = loaded_model(**inputs)

            # Process batch with confidence
            probs = softmax(outputs.logits, dim=1)
            confidences, pred_indices = torch.max(probs, dim=1)
            predictions = label_encoder.inverse_transform(pred_indices.cpu().numpy())

            results.extend([
                {
                    'prediction': pred,
                    'confidence': round(conf.item(), 4)
                }
                for pred, conf in zip(predictions, confidences)
            ])

    return results

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
result = predict("i have issue in my fammily")
print(f"Prediction: {result['prediction']} (Confidence: {result['confidence']:.2%})")

NameError: name 'predict' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from tqdm.auto import tqdm
import zipfile
import os
from google.colab import drive
def fast_zip_with_progress(source_dir, output_path):
    files_to_zip = []
    for root, _, files in os.walk(source_dir):
        for file in files:
            files_to_zip.append(os.path.join(root, file))

    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in tqdm(files_to_zip, unit='file', desc='Compressing'):
            arcname = os.path.relpath(file, source_dir)
            zipf.write(file, arcname)

fast_zip_with_progress('model_assets', '/content/drive/MyDrive/Fast_genre.zip')

Compressing:   0%|          | 0/6 [00:00<?, ?file/s]

In [ ]:
!du -sh model_assets/

du: cannot access 'model_assets/': No such file or directory


read weights file after compress


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
import zipfile
import os

# Path to the ZIP file in your Drive
zip_path = '/content/drive/MyDrive/Fast_genre.zip'

# Destination folder
output_dir = '/content/model_assets'
os.makedirs(output_dir, exist_ok=True)


In [ ]:
from tqdm import tqdm

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()

    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(member=file, path=output_dir)

Extracting files: 100%|██████████| 6/6 [00:26<00:00,  4.38s/it]


In [ ]:
import gdown
import zipfile
import os
from tqdm import tqdm

# Step 1: Download the ZIP file from Google Drive (shared link)
file_id = '1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR'  # Extracted from shared URL
zip_path = 'downloaded_file.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', zip_path, quiet=False)

# Step 2: Create the output directory
output_dir = 'model_assets'
os.makedirs(output_dir, exist_ok=True)

# Step 3: Unzip the file with progress
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    for file in tqdm(file_list, desc="Extracting files"):
        zip_ref.extract(member=file, path=output_dir)

print(f"\n✅ Done! Files extracted to '{output_dir}'")


Downloading...
From (original): https://drive.google.com/uc?id=1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR
From (redirected): https://drive.google.com/uc?id=1juliMp2yJwMe08Yh0s2uYpu_3U4miUAR&confirm=t&uuid=08259068-767a-4e8f-8a38-87845763c6c5
To: /content/downloaded_file.zip
100%|██████████| 1.05G/1.05G [00:13<00:00, 80.3MB/s]
Extracting files: 100%|██████████| 6/6 [00:22<00:00,  3.80s/it]


✅ Done! Files extracted to 'model_assets'
